In [38]:
import pandas as pd
import numpy as np

# Imports données

In [40]:
application = pd.read_csv("../data/application_train_vf.csv", sep=",", index_col=0)
bureau = pd.read_csv("../data/bureau.csv", sep=",")
POS_CASH_balance = pd.read_csv("../data/POS_CASH_balance.csv", sep=",")
credit_card_balance = pd.read_csv("../data/credit_card_balance.csv", sep=",")
previous_application = pd.read_csv("../data/previous_application.csv", sep=",")
installments_payments = pd.read_csv("../data/installments_payments.csv", sep=",")

# Restriction aux variables que l'on a ciblé

In [41]:
cols_from_application = ["SK_ID_CURR", 'TARGET',"date_mensuelle" ,"NAME_CONTRACT_TYPE", "AMT_INCOME_TOTAL", "AMT_CREDIT",
                         "AMT_ANNUITY", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE",
                         "NAME_FAMILY_STATUS",'NAME_HOUSING_TYPE', 'DAYS_EMPLOYED',
                         "ORGANIZATION_TYPE", "EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]

cols_from_bureau = ["SK_ID_CURR","AMT_CREDIT_SUM", "AMT_CREDIT_SUM_DEBT", "AMT_CREDIT_SUM_LIMIT",
                    "AMT_CREDIT_SUM_OVERDUE", "CREDIT_TYPE", "AMT_ANNUITY"]

cols_from_POS_CASH_balance = ["SK_ID_CURR", "CNT_INSTALMENT_FUTURE"]

cols_from_credit_card_balance = ["SK_ID_CURR","AMT_BALANCE", "AMT_DRAWINGS_CURRENT", "AMT_PAYMENT_CURRENT"]

cols_from_previous_application = ['SK_ID_CURR',"AMT_APPLICATION", "AMT_CREDIT", "RATE_INTEREST_PRIMARY",
                                  "NAME_CLIENT_TYPE", "NFLAG_INSURED_ON_APPROVAL"]

cols_from_installments_payments = ['SK_ID_CURR',"AMT_PAYMENT"]

In [42]:
application = application[cols_from_application]
bureau = bureau[cols_from_bureau]
POS_CASH_balance = POS_CASH_balance[cols_from_POS_CASH_balance]
credit_card_balance = credit_card_balance[cols_from_credit_card_balance]
previous_application = previous_application[cols_from_previous_application]
installments_payments = installments_payments[cols_from_installments_payments]

In [43]:
def treatement_NaN_values(df, NaN_threshold):
    for cols in df.select_dtypes(include=np.number).columns:

        if df[cols].isna().sum()/df.shape[0]*100 > NaN_threshold: df.drop(cols, axis=1, inplace=True)
        else : df.fillna({cols: df[cols].median()}, inplace=True)

    for cols in df.select_dtypes(include="object").columns:
        if df[cols].isna().sum()/df.shape[0]*100 > NaN_threshold: df.drop(cols, axis=1, inplace=True)
        else : df.fillna({cols: df[cols].mode()[0]}, inplace=True)

In [44]:
# application
treatement_NaN_values(df=application, NaN_threshold=35)

# bureau
treatement_NaN_values(df=bureau, NaN_threshold=35)

# POS_CASH_balance
treatement_NaN_values(df=POS_CASH_balance, NaN_threshold=35)

# credit_card_balance
treatement_NaN_values(df=credit_card_balance, NaN_threshold=35)

# previous_application
treatement_NaN_values(df=previous_application, NaN_threshold=35)

# installments_payments
treatement_NaN_values(df=installments_payments, NaN_threshold=35)

## fonctions pour le traitement des données ici

In [45]:
def treatement_datas_with_numbers_and_categoricals(df, list_cols, categorical_col):
    df_temp1 = df.groupby('SK_ID_CURR')[list_cols].mean()

    compte_par_id = df.groupby(['SK_ID_CURR', categorical_col]).size().reset_index(name='Nombre_Occurrences')

    df_sorted = compte_par_id.sort_values(by=['SK_ID_CURR', 'Nombre_Occurrences', categorical_col], ascending=[True, False, True])

    df_temp2 = df_sorted.drop_duplicates(subset=['SK_ID_CURR'], keep='first')

    df = df_temp1.merge(df_temp2, on='SK_ID_CURR')
    df.drop("Nombre_Occurrences", axis=1, inplace=True)
    return df

In [ ]:
def treatement_datas_with_numbers_only(df, cols):
    if isinstance(cols, str) :
        df = df.groupby('SK_ID_CURR')[cols].mean().reset_index(name=cols)
    elif isinstance(cols, list):
        df = df.groupby("SK_ID_CURR")[cols].mean()
    return df

# Bureau

In [46]:
bureau.head()

,SK_ID_CURR,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE
0,215354,91323.0,0.0,0.0,0.0,Consumer credit
1,215354,225000.0,171342.0,0.0,0.0,Credit card
2,215354,464323.5,0.0,0.0,0.0,Consumer credit
3,215354,90000.0,0.0,0.0,0.0,Credit card
4,215354,2700000.0,0.0,0.0,0.0,Consumer credit


In [47]:
bureau = treatement_datas_with_numbers_and_categoricals(
    df=bureau,
    list_cols=["AMT_CREDIT_SUM", "AMT_CREDIT_SUM_DEBT", 'AMT_CREDIT_SUM_LIMIT', "AMT_CREDIT_SUM_OVERDUE"],
    categorical_col="CREDIT_TYPE")
bureau.head()

,SK_ID_CURR,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE
0,100001,207623.571429,85240.928571,0.000000,0.0,Consumer credit
1,100002,108131.945625,30722.625000,3998.570625,0.0,Consumer credit
2,100003,254350.125000,0.000000,202500.000000,0.0,Consumer credit
3,100004,94518.900000,0.000000,0.000000,0.0,Consumer credit
4,100005,219042.000000,189469.500000,0.000000,0.0,Consumer credit


# POS_CASH_BALANCE

In [48]:
POS_CASH_balance.fillna({"CNT_INSTALMENT_FUTURE": POS_CASH_balance["CNT_INSTALMENT_FUTURE"].median()}, inplace=True)

In [49]:
POS_CASH_balance

,SK_ID_CURR,CNT_INSTALMENT_FUTURE
0,182943,45.0
1,367990,35.0
2,397406,9.0
3,269225,42.0
4,334279,35.0
...,...,...
10001353,226558,0.0
10001354,141565,0.0
10001355,315695,0.0
10001356,450255,0.0


In [50]:
POS_CASH_balance = treatement_datas_with_numbers_only(df=POS_CASH_balance, cols="CNT_INSTALMENT_FUTURE")
POS_CASH_balance.head()

,SK_ID_CURR,CNT_INSTALMENT_FUTURE
0,100001,1.444444
1,100002,15.000000
2,100003,5.785714
3,100004,2.250000
4,100005,7.181818


# credit_card_balance

In [51]:
credit_card_balance = treatement_datas_with_numbers_only(df=credit_card_balance,
                                                         cols=['SK_ID_CURR','AMT_BALANCE', 'AMT_DRAWINGS_CURRENT', 'AMT_PAYMENT_CURRENT'])
credit_card_balance.head()

,SK_ID_CURR,AMT_BALANCE,AMT_DRAWINGS_CURRENT,AMT_PAYMENT_CURRENT
SK_ID_CURR,,,,
100006,100006.0,0.000000,0.000000,2702.700000
100011,100011.0,54482.111149,2432.432432,4843.064189
100013,100013.0,18159.919219,5953.125000,7168.346250
100021,100021.0,0.000000,0.000000,2702.700000
100023,100023.0,0.000000,0.000000,2702.700000


# previous_application

In [52]:
previous_application = treatement_datas_with_numbers_and_categoricals(df=previous_application,
                                                                      list_cols=['AMT_APPLICATION', "AMT_CREDIT"],
                                                                      categorical_col="NAME_CLIENT_TYPE")
previous_application.head()

,SK_ID_CURR,AMT_APPLICATION,AMT_CREDIT,NAME_CLIENT_TYPE
0,100001,24835.50,23787.00,Refreshed
1,100002,179055.00,179055.00,New
2,100003,435436.50,484191.00,Refreshed
3,100004,24282.00,20106.00,New
4,100005,22308.75,20076.75,New


# installments_payments

In [53]:
installments_payments = treatement_datas_with_numbers_only(df=installments_payments, cols="AMT_PAYMENT")
installments_payments.head()

,SK_ID_CURR,AMT_PAYMENT
0,100001,5885.132143
1,100002,11559.247105
2,100003,64754.586000
3,100004,7096.155000
4,100005,6240.205000


# MERGING

In [62]:
def keep_only_rows_with_ids(df, ids):
    df = df[df["SK_ID_CURR"].isin(ids)]
    df.reset_index(drop=True, inplace=True)
    return df

In [54]:
ids = application["SK_ID_CURR"]

In [55]:
bureau = keep_only_rows_with_ids(bureau, ids)
POS_CASH_balance = keep_only_rows_with_ids(POS_CASH_balance, ids)
credit_card_balance = keep_only_rows_with_ids(credit_card_balance, ids)
previous_application = keep_only_rows_with_ids(previous_application, ids)
installments_payments = keep_only_rows_with_ids(installments_payments, ids)

In [57]:
temp1 = application.merge(bureau, on='SK_ID_CURR', how='left')
temp2 = temp1.merge(POS_CASH_balance, on='SK_ID_CURR', how='left')
temp3 = temp2.merge(credit_card_balance, on='SK_ID_CURR', how='left')
temp4 = temp3.merge(previous_application, on='SK_ID_CURR', how='left')
data = temp4.merge(installments_payments, on='SK_ID_CURR', how='left')
data.head()

,SK_ID_CURR,TARGET,date_mensuelle,NAME_CONTRACT_TYPE,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,...,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,CNT_INSTALMENT_FUTURE,AMT_BALANCE,AMT_DRAWINGS_CURRENT,AMT_PAYMENT_CURRENT,AMT_APPLICATION,AMT_CREDIT_y,NAME_CLIENT_TYPE,AMT_PAYMENT
0,333721,0,2013-01-01,Cash loans,292500.0,1102500.0,32364.0,Working,Secondary / secondary special,Married,...,0.0,Consumer credit,5.894737,NaN,NaN,NaN,84628.5,87904.50,Repeater,16194.380625
1,250254,0,2013-01-01,Cash loans,180000.0,270000.0,10179.0,Working,Higher education,Married,...,0.0,Consumer credit,11.112676,NaN,NaN,NaN,48510.9,50020.65,Repeater,5617.057826
2,265071,0,2013-01-01,Cash loans,67500.0,80865.0,7546.5,Working,Secondary / secondary special,Married,...,0.0,Consumer credit,6.636364,NaN,NaN,NaN,42750.0,47263.50,New,4260.588750
3,227569,0,2013-01-01,Cash loans,157500.0,1078200.0,38331.0,Working,Secondary / secondary special,Separated,...,0.0,Consumer credit,5.000000,NaN,NaN,NaN,34596.0,38943.00,New,4301.827500
4,212616,0,2013-01-01,Cash loans,112500.0,225000.0,15165.0,Working,Secondary / secondary special,Civil marriage,...,0.0,Consumer credit,6.000000,NaN,NaN,NaN,71032.5,68175.00,New,7946.006250


In [63]:
def treatment_NaN_in_merge_data(df, value_target):
    df_target = df[df["TARGET"]==value_target]
    for col in df_target.select_dtypes(include=np.number).columns:
        # print(col,":", df_target[col].isna().sum()/df_target.shape[0]*100)
        if df_target[col].isna().sum()/df_target.shape[0]*100 > 35: df_target.drop(col, axis=1, inplace=True)
        else : df_target.fillna({col: df_target[col].median()}, inplace=True)
    for col in df_target.select_dtypes(include="object").columns:
        if df_target[col].isna().sum()/df_target.shape[0]*100 > 35: df_target.drop(col, axis=1, inplace=True)
        else : df_target.fillna({col: df_target[col].mode()}, inplace=True)
    return df_target

In [59]:
data_target_1 = treatment_NaN_in_merge_data(df=data, value_target=1)
data_target_0 = treatment_NaN_in_merge_data(df=data, value_target=0)



/var/folders/cs/hh3_26_57sx7l8sdk574zbdh0000gn/T/ipykernel_26540/3881792063.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else : df_target.fillna({col: df_target[col].median()}, inplace=True)
/var/folders/cs/hh3_26_57sx7l8sdk574zbdh0000gn/T/ipykernel_26540/3881792063.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else : df_target.fillna({col: df_target[col].median()}, inplace=True)
/var/folders/cs/hh3_26_57sx7l8sdk574zbdh0000gn/T/ipykernel_26540/3881792063.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyd

In [64]:
def final_table(data_target_1, data_target_0):
    data_final = pd.concat([data_target_1, data_target_0])
    data_final["date_mensuelle"] = pd.to_datetime(data_final["date_mensuelle"])
    data_final.sort_values(by="date_mensuelle", ascending=True, inplace=True)
    return data_final

In [61]:
data_final = final_table(data_target_1, data_target_0)
data_final.head()

,SK_ID_CURR,TARGET,date_mensuelle,NAME_CONTRACT_TYPE,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,...,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,CNT_INSTALMENT_FUTURE,AMT_APPLICATION,AMT_CREDIT_y,NAME_CLIENT_TYPE,AMT_PAYMENT
848,341177,1,2013-01-01,Cash loans,99000.0,354276.0,28120.5,Working,Secondary / secondary special,Civil marriage,...,52490.250000,0.000000,0.0,0.0,Consumer credit,5.000000,65316.150,69522.3000,Repeater,4164.552692
2205,329794,0,2013-01-01,Cash loans,270000.0,966645.0,38466.0,Working,Higher education,Single / not married,...,488655.189000,180000.900000,0.0,0.0,Consumer credit,9.550000,78997.500,78005.2500,Repeater,5928.888140
2206,410860,0,2013-01-01,Cash loans,157500.0,225000.0,9909.0,Pensioner,Higher education,Married,...,153699.367500,3730.875000,0.0,0.0,Consumer credit,5.000000,167314.500,179275.5000,New,19716.777000
2207,125382,0,2013-01-01,Cash loans,76500.0,808650.0,23773.5,Pensioner,Secondary / secondary special,Married,...,192250.500000,47592.000000,0.0,0.0,Consumer credit,6.785714,170644.500,168273.0000,New,12352.523824
2208,416718,0,2013-01-01,Cash loans,157500.0,315000.0,11673.0,Working,Secondary / secondary special,Married,...,418197.310312,202930.281562,0.0,0.0,Consumer credit,8.707692,97378.875,118605.9375,Repeater,27195.907846


In [ ]:
# data_final.to_csv("../data/data_variables_metiers.csv", index=False)